## Import Packages


In [1]:
import numpy as np
from PIL import Image
import cv2
import time


In [2]:
def SAD(R, T, i, j, x, y, n):
    dist = np.sum(np.abs(T[x:x+n, y:y+n, :].astype(int) - R[i:i+n, j:j+n, :].astype(int)))
    return dist

def PSNR(img, new_img):
    mse = np.mean((img.astype(np.float64) / 255 -
                  new_img.astype(np.float64) / 255) ** 2)
    return 10 * np.log10(1. / mse)


### 1. Search Methods

In [3]:
# p: search range, n: macroblock size
def fs(R, T, p, n):
    motion_vectors = np.zeros((T.shape[0]//n, T.shape[1]//n, 2))

    totalSAD = 0
    for i in range(0, T.shape[0], n): #height
        for j in range(0, T.shape[1], n): # weight
            min_dist = np.inf
            mv = np.zeros((2,))
            # compute SAD
            for k in range(max(i-p, 0), min(i+p, R.shape[0]-n)):
                for l in range(max(j-p, 0), min(j+p, R.shape[1]-n)):
                    dist = SAD(R, T, k, l, i, j, n)
                    # Update the minimum distance
                    if dist < min_dist:
                        min_dist = dist
                        mv[0] = l- j
                        mv[1] = k - i
            totalSAD += min_dist
            # Store the motion vector for this block
            motion_vectors[i//n, j//n, :] = mv

    return motion_vectors, totalSAD


In [4]:
# p: search range
# n: macroblock size
def twodLOG(R, T, p, n):
    motion_vectors = np.zeros((T.shape[0]//n, T.shape[1]//n, 2))
    totalSAD = 0
    for i in range(0, T.shape[0], n):
        for j in range(0, T.shape[1], n):
            mv = np.zeros((2,))
            step = np.floor(np.log2(p))
            NN = max(2, 2**(step-1))
            cx = cy = 0
            Mset = np.array([[0, 0], [NN, 0], [0, NN], [-NN,  0], [0, -NN]])
            lowerX, upperX = max(j-p, 0), min(j+p, R.shape[1]-n)
            lowerY, upperY = max(i-p, 0), min(i+p, R.shape[0]-n)

            while (NN != 1):
                min_dist = np.inf
                for each in Mset:
                    k, l = int(each[0]), int(each[1])
                    x = max(lowerX, min(upperX, j+cx+k))
                    y = max(lowerY, min(upperY, i+cy+l))
            
                    dist = SAD(R, T, y, x, i, j, n)
                    # Update the minimum distance
                    if dist < min_dist:
                        min_dist = dist
                        mv[0] = x - j
                        mv[1] = y - i
                        best = (k, l)
          
                if best == (0, 0): NN /= 2
                else:
                    cx = cx + best[0]
                    cy = cy + best[1]
            
            FinalSet = [-1, 0, 1]
            for l in range(3):
                for k in range(3):
                    x = max(lowerX, min(upperX, j+cx+FinalSet[k]))
                    y = max(lowerY, min(upperY, i+cy+FinalSet[l]))
            
                    dist = SAD(R, T, y, x, i, j, n)
                    if dist < min_dist:
                        min_dist = dist
                        mv[0] = x - j
                        mv[1] = y - i
            totalSAD += min_dist
            # Store the motion vector for this block
            motion_vectors[i//n, j//n, :] = mv

    return motion_vectors, totalSAD


### Output result

In [5]:
P = [8, 8, 16, 16]
N = [8, 16, 8, 16]
predicted_path = ['out/full_predicted_r8_b8.jpg', 'out/full_predicted_r8_b16.jpg',
                'out/full_predicted_r16_b8.jpg', 'out/full_predicted_r16_b16.jpg']
mv_path = ['out/full_motion_vector_r8_b8.jpg', 'out/full_motion_vector_r8_b16.jpg',
                'out/full_motion_vector_r16_b8.jpg', 'out/full_motion_vector_r16_b16.jpg']
residual_output = ['out/full_residual_r8_b8.jpg', 'out/full_residual_r8_b16.jpg',
                    'out/full_residual_r16_b8.jpg', 'out/full_residual_r16_b16.jpg']

ref_img = Image.open('img/40.jpg')
ref_img_arr = np.array(ref_img)
target_img = Image.open('img/42.jpg')
target_img_arr = np.array(target_img)

for t in range(4):
    n = N[t]
    p = P[t]
    motion_vectors, totalSAD = fs(ref_img_arr, target_img_arr, p, n)
    img = cv2.imread('img/42.jpg')
    # Perform motion compensation

    predicted = np.zeros_like(target_img_arr)
    for i in range(motion_vectors.shape[0]):
        for j in range(motion_vectors.shape[1]):
            mv = motion_vectors[i, j]
            ref_i = int(i*n + mv[1])
            ref_j = int(j*n + mv[0])
            predicted[i*n:(i+1)*n, j*n:(j+1)*n, :] = ref_img_arr[ref_i:ref_i+n, ref_j:ref_j+n, :]
            
            start_point = (ref_j, ref_i)
            end_point = (j*n, i*n)
            img = cv2.arrowedLine(img, start_point, end_point, (0, 0, 255), thickness=1, tipLength=0.4)

    # predicted image
    new_img = Image.fromarray(predicted)
    new_img.save(predicted_path[t])
    # Display the image with the motion vectors drawn on it
    cv2.imwrite(mv_path[t], img)
    # Residual image
    residual_img_arr = np.subtract(target_img_arr.astype(np.float64), predicted.astype(np.float64))
    residual_img_arr = np.clip(residual_img_arr, 0, 255).astype(np.uint8)
    residual_img = Image.fromarray(residual_img_arr)
    residual_img.save(residual_output[t])

    print(predicted_path[t], ', total SAD:', totalSAD, ', PSNR:', PSNR(target_img_arr, predicted))


out/full_predicted_r8_b8.jpg , total SAD: 15680626 , PSNR: 24.03669870020936
out/full_predicted_r8_b16.jpg , total SAD: 17546298 , PSNR: 23.31845360732124
out/full_predicted_r16_b8.jpg , total SAD: 11707596 , PSNR: 27.149372134942695
out/full_predicted_r16_b16.jpg , total SAD: 13648410 , PSNR: 25.996482636118458


In [6]:
P = [8, 8, 16, 16]
N = [8, 16, 8, 16]
predicted_path = ['out/2d_predicted_r8_b8.jpg', 'out/2d_predicted_r8_b16.jpg',
               'out/2d_predicted_r16_b8.jpg', 'out/2d_predicted_r16_b16.jpg']
mv_path = ['out/2d_motion_vector_r8_b8.jpg', 'out/2d_motion_vector_r8_b16.jpg',
                'out/2d_motion_vector_r16_b8.jpg', 'out/2d_motion_vector_r16_b16.jpg']
residual_output = ['out/2d_residual_r8_b8.jpg', 'out/2d_residual_r8_b16.jpg',
                    'out/2d_residual_r16_b8.jpg', 'out/2d_residual_r16_b16.jpg']

ref_img = Image.open('img/40.jpg')
ref_img_arr = np.array(ref_img)
target_img = Image.open('img/42.jpg')
target_img_arr = np.array(target_img)

for t in range(4):
    n = N[t]
    p = P[t]
    motion_vectors, totalSAD = twodLOG(ref_img_arr, target_img_arr, p, n)
    img = cv2.imread('img/42.jpg')
    
    predicted = np.zeros_like(target_img_arr)
    for i in range(motion_vectors.shape[0]):
        for j in range(motion_vectors.shape[1]):
            mv = motion_vectors[i, j]
            ref_i = int(i * n + mv[1])
            ref_j = int(j * n + mv[0])
            predicted[i*n:(i+1)*n, j*n:(j+1)*n,:] = ref_img_arr[ref_i:ref_i+n, ref_j:ref_j+n, :]

            # Calculate the start and end points of the arrowed line for the current motion vector
            start_point = (ref_j, ref_i)
            end_point = (j*n, i*n)
            img = cv2.arrowedLine(img, start_point, end_point, (0, 0, 255), thickness=1, tipLength=0.4)

    # Predicted image
    new_img = Image.fromarray(predicted)
    new_img.save(predicted_path[t])
    # Display the image with the motion vectors drawn on it
    cv2.imwrite(mv_path[t], img)
    # Residual image
    residual_img_arr = np.subtract(target_img_arr.astype(np.float64), predicted.astype(np.float64))
    residual_img_arr = np.clip(residual_img_arr, 0, 255).astype(np.uint8)
    residual_img = Image.fromarray(residual_img_arr)
    residual_img.save(residual_output[t])
    
    print(predicted_path[t], ', total SAD:', totalSAD, ', PSNR:', PSNR(target_img_arr, predicted))

out/2d_predicted_r8_b8.jpg , total SAD: 19725072 , PSNR: 23.645437226243807
out/2d_predicted_r8_b16.jpg , total SAD: 20972545 , PSNR: 23.119465277820023
out/2d_predicted_r16_b8.jpg , total SAD: 20011768 , PSNR: 24.73282826103294
out/2d_predicted_r16_b16.jpg , total SAD: 21011795 , PSNR: 24.486959123063595


### 2. Experiment

In [22]:
ref_img_arr = np.array(Image.open('img/40.jpg'))
target_img_arr = np.array(Image.open('img/51.jpg'))
p, n = 8, 8
motion_vectors, totalSAD = fs(ref_img_arr, target_img_arr, p, n)
# Perform motion compensation
predicted = np.zeros_like(target_img_arr)
for i in range(motion_vectors.shape[0]):
    for j in range(motion_vectors.shape[1]):
        mv = motion_vectors[i, j]
        ref_i = int(i * n + mv[1])
        ref_j = int(j * n + mv[0])
        predicted[i*n:(i+1)*n, j*n:(j+1)*n,:] = ref_img_arr[ref_i:ref_i+n, ref_j:ref_j+n, :]
print('SAD:', totalSAD)
print('PSNR:', PSNR(target_img_arr, predicted))

SAD: 30820829
PSNR: 21.325464005500194


### 3. Analyze the time complexity


In [23]:
ref_img_arr = np.array(Image.open('img/40.jpg'))
target_img_arr = np.array(Image.open('img/42.jpg'))
p, n = 8, 8
start_time = time.time()
motion_vectors, totalSAD = fs(ref_img_arr, target_img_arr, p, n)
print("fs: %s seconds" % (time.time() - start_time))
start_time = time.time()
motion_vectors, totalSAD = twodLOG(ref_img_arr, target_img_arr, p, n)
print("2D log: %s seconds" % (time.time() - start_time))

fs: 32.88054633140564 seconds
2D log: 4.186006307601929 seconds


In [24]:
ref_img_arr = np.array(Image.open('img/40.jpg'))
target_img_arr = np.array(Image.open('img/42.jpg'))
p, n = 16, 8
start_time = time.time()
motion_vectors, totalSAD = fs(ref_img_arr, target_img_arr, p, n)
print("fs: %s seconds" % (time.time() - start_time))
start_time = time.time()
motion_vectors, totalSAD = twodLOG(ref_img_arr, target_img_arr, p, n)
print("2D log: %s seconds" % (time.time() - start_time))

fs: 139.93482899665833 seconds
2D log: 5.121671438217163 seconds
